<a href="https://colab.research.google.com/github/teddylew12/Waymo_Open_DataSet/blob/master/Generate_Weather_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/data')


In [0]:
!pip3 install waymo-open-dataset
!pip3 install tensorflow==2.0.0
import os
import tensorflow as tf
import math
import numpy as np
import itertools
import random
from PIL import Image
from tqdm import tqdm
import pickle

from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset


In [0]:
# Change the CD To your specific path
DIRNAME ="path/to/file"
os.chdir(DIRNAME)
! mkdir faster_images

In [0]:
def save_image(image,savename):
    
    decoded=tf.image.decode_jpeg(image)
    tonumpy=decoded.numpy()
    im=Image.fromarray(tonumpy)
    im.save(os.path.join('faster_images/', name))

def make_roi_dict(lbl, filename): 
    
    labelsMsg.CopyFrom(lbl)
    boxes = np.array([[label.box.center_x - 0.5 * label.box.length,
                          label.box.center_y - 0.5 * label.box.width,
                          label.box.center_x + 0.5 * label.box.length,
                          label.box.center_y + 0.5 * label.box.width] for label in lbl.labels])
    labels = np.array([label.type+1 for label in labelsMsg.labels])
    return {'file_name':filename,'boxes':boxes, 'class':labels, 'is_crowd':False}
def extract_data_name(filename):
    '''
    Extracts Unique Identifier in each file name
    Before:segment-15868625208244306149_4340_000_4360_000_with_camera_labels
    After:15868625208244306149_4340_000_4360_000
    '''
    return filename.split("-")[-1][:-28]
def create_list_of_roi_dicts(dirname):
    
    roi_list = []
    with os.scandir(dirname) as scanobj:
        for entry in scanobj:
            if entry.name.endwith('.tfrecord'):
                dataset = tf.data.TFRecordDataset(entry.path)
                d_name = extract_data_name(entry.name)
                #Keep track of which frame in the data object is being processed
                frame_num = 0
                for data in dataset:
                    frame = open_dataset.Frame()
                    frame.ParseFromString(bytearray(data.numpy()))

                    #Keep track of which camera image of the frame is being processed
                    image_num = 0

                    for lbl, img in zip(frame.camera_labels, frame.images):
                        
                        name=f"{d_name}_frame_{frame_num}_image_{image_num}.png"
                        roi_list.append(make_roi_dict(lbl, name))
                        save_name(img.image, name)
                        image_num += 1
                    
                    frame_num += 1
    return roi_list
def create_dataset(dirname):
    rois = create_list_of_roi_dicts(dirname)
    with open("faster_rcnn_rois.pickle",'wb') as fileObj:
        pickle.dump(p,fileObj)
        
DIRNAME= ""
create_dataset(dirname)